In [2]:
import numpy as np
import scipy.stats as sc
import matplotlib.pyplot as plt
from time import time

In [3]:
# ground truth data
ground_truths = np.load('mt_trajectories.npy')
ground_truths

array([[[ -8.07218076,  -4.49125581,  -0.91033086,   2.67059409,
           6.25151904,   9.83244399,  13.41336894,  16.99429389,
          20.57521884,  24.15614379],
        [ -0.8844742 ,  -2.77044353,  -4.65641285,  -6.54238217,
          -8.4283515 , -10.31432082, -12.20029014, -14.08625947,
         -15.97222879, -17.85819811],
        [ -3.88086752,  -2.92152956,  -1.96219159,  -1.00285363,
          -0.04351567,   0.91582229,   1.87516025,   2.83449821,
           3.79383618,   4.75317414]],

       [[ -6.46454434,  -3.36883308,  -0.27312181,   2.82258945,
           5.91830072,   9.01401198,  12.10972325,  15.20543452,
          18.30114578,  21.39685705],
        [  5.1703906 ,   6.05898913,   6.94758765,   7.83618618,
           8.72478471,   9.61338324,  10.50198177,  11.3905803 ,
          12.27917883,  13.16777736],
        [ -1.3581392 ,  -0.35850924,   0.64112072,   1.64075068,
           2.64038064,   3.6400106 ,   4.63964056,   5.63927052,
           6.63890048,   7.6